In [16]:
import pandas as pd 
import plotly.express as px
import dash
from dash import dcc
from dash import html
import plotly.graph_objs as go

# We need cufflinks to link plotly to pandas and add the iplot method
import cufflinks as cf
cf.go_offline()

In [2]:
data = pd.read_csv('IMDb cleaned data.csv',low_memory = False)

In [3]:
data.head()

,imdb_title_id,title,date_published,year,genre,duration,country,director,votes,language
0,tt0002101,Cleopatra,1912-11-13,1912,"Drama,History",100,USA,Charles L. Gaskill,446,English
1,tt0002130,L'Inferno,1911-03-06,1911,"Adventure,Drama,Fantasy",68,Italy,"Francesco Bertolini, Adolfo Padovan",2237,Italian
2,tt0002423,Madame DuBarry,1919-11-26,1919,"Biography,Drama,Romance",85,Germany,Ernst Lubitsch,753,German
3,tt0002445,Quo Vadis?,1913-03-01,1913,"Drama,History",120,Italy,Enrico Guazzoni,273,Italian
4,tt0002461,Richard III,1912-10-15,1912,Drama,55,"France, USA","André Calmettes, James Keane",225,English


In [4]:
#Select = data[data['year'] == 1944]['country'].str.cat(sep = ',').replace(" ","").lower().replace("westgermany","germany")
Select = data['country'].str.cat(sep = ',').replace(" ","").lower().replace("westgermany","germany")
G = pd.Series(Select.split(','))
frame = G.value_counts().rename_axis('country').reset_index(name='Frequency')
frame['country'] = frame['country'].str.capitalize() 

In [5]:
fig = px.scatter_geo(frame, locations="country", color_discrete_sequence=["orange"], 
                     hover_name="country", size="Frequency", locationmode='country names',
                     projection="natural earth")
fig.show()

In [7]:
years = data.drop_duplicates(subset = "year")
years = sorted(years['year'])

In [1]:
app = dash.Dash()

app.layout = html.Div([
    html.H2(children="Let's Explore Movies Data!!", style={'textAlign':'center'}),
    # open container
    html.Div([
        dcc.RadioItems(
            # id = identifing dash components
            id = 'Selection',
            # options = list of dictionaries
            options = [{'label': i, 'value': i} for i in ['Genre', 'Country', 'Language']],
            # value = intial selected item
            value = 'Genre',
            # 'inline-block' for horizontal
            labelStyle = {'display': 'inline-block'}
        ),
        
        dcc.Dropdown(
            id = 'drop',
            options = [{'label': i, 'value': i} for i in years],
            value = 'All Years',
            # for choosing multiple values
            multi = True,
            # placeholder = defining default text to show
            placeholder = "Select a multiple  years"
         ),
        
        dcc.Graph(id = 'indicator_graphic_1')
    ]),
    # ************************************************************************************ #    
    html.Div([
        html.H2(children="Duration changes", style={'textAlign':'center'}),

        dcc.RangeSlider(
            id = 'slider_1',
            min = min(years),
            max = max(years),
            value = [min(years), max(years)],
            tooltip = {"placement": "bottom", "always_visible": True},
            pushable = True,
            dots = False
        ),
        
        html.Div(id='output_range_1', style={'textAlign':'center'}),
        
        dcc.Graph(id = 'indicator_graphic_2')
    ]),
    # ************************************************************************************ #
    html.Div([
        html.H2(children="Distribution of the most productive Movies", style={'textAlign':'center'}),
        
        dcc.RangeSlider(
            id = 'slider_2',
            min = min(years),
            max = max(years),
            value = [min(years), max(years)],
            tooltip = {"placement": "bottom", "always_visible": True},
            allowCross = False,
            dots = False
        ),
        
        html.Div(id='output_range_2', style={'textAlign':'center'}),
        
        dcc.Graph(id = 'indicator_graphic_3')
    ])
    
])

# Bar Plot
@app.callback(
    dash.dependencies.Output('indicator_graphic_1', 'figure'),
    dash.dependencies.Input('Selection', 'value'),
    dash.dependencies.Input('drop', 'value'))
def update_graph(Selection, year):
    Selection = Selection.lower()
    if year == "All Years" or len(year) == 0:
        year = "All Years"
        Select = data[Selection].str.cat(sep = ',')
        G = pd.Series(Select.split(','))
        frame = G.value_counts().rename_axis(Selection).reset_index(name='Frequency')
        
    else: 
        df = data[data['year'].isin(year)]
        Select = df[Selection].str.cat(sep = ',')
        G = pd.Series(Select.split(','))
        frame = G.value_counts().rename_axis(Selection).reset_index(name='Frequency')
    
    return { 'data': [ go.Bar(x = frame[Selection],y = frame['Frequency'])], 
            'layout': go.Layout(title='The most trend {} in : {}'.format(Selection, year), 
                                xaxis={'title': Selection},
                                yaxis={'title':'Frequency'}) }


# Line Plot
@app.callback(
    dash.dependencies.Output('output_range_1', 'children'),
    dash.dependencies.Input('slider_1', 'value'))
def update_output(value):
    return 'Duration change from {} to {}'.format( value[0], value[1])

@app.callback(
    dash.dependencies.Output('indicator_graphic_2', 'figure'),
    dash.dependencies.Input('slider_1', 'value'))
def update_graph(value):
    mean_of_duration = data.groupby(['year'])['duration'].mean().reset_index() 
    df = mean_of_duration[mean_of_duration['year'].isin([x for x in range(value[0], value[1]+1)])]
    return px.line(df, x="year", y="duration")


# Bubble Map
@app.callback(
    dash.dependencies.Output('output_range_2', 'children'),
    dash.dependencies.Input('slider_2', 'value'))
def update_output(value):
    return 'Distribution of the most movie production countries from {} to {}'.format( value[0], value[1])


@app.callback(
    dash.dependencies.Output('indicator_graphic_3', 'figure'),
    dash.dependencies.Input('slider_2', 'value'))
def update_graph(value):
    Select = data[data['year'].isin(value)]['country'].str.cat(sep = ',').replace(" ","").lower().replace("westgermany","germany")
    G11 = pd.Series(Select.split(','))
    frame = G11.value_counts().rename_axis('country').reset_index(name='Frequency')
    frame['country'] = frame['country'].str.capitalize()     
    return px.scatter_geo(frame, locations="country", color_discrete_sequence=["orange"], 
                         hover_name="country", size="Frequency", locationmode='country names',
                         projection="natural earth")

NameError: name 'dash' is not defined

In [9]:
if __name__ == "__main__":
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Jan/2022 19:31:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2022 19:31:15] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2022 19:31:15] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2022 19:31:15] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [02/Jan/2022 19:31:15] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [02/Jan/2022 19:31:15] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [02/Jan/2022 19:31:15] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [02/Jan/2022 19:31:15] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2022 19:31:15] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2022 19:31:15] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2022 19:31:15] "POST /_dash-